In [ ]:
!nvidia-smi     

: 

In [ ]:
a=6

: 

In [1]:
import torch

In [2]:
a = 1e-12

In [3]:
type(a)

float

In [5]:
torch.tensor(a)

tensor(1.0000e-12)

In [20]:
a = torch.rand(2,9,786).float()

In [10]:
a.shape

torch.Size([2, 9, 786])

In [18]:
from torch import nn
ln = nn.LayerNorm(786, eps=1e-12).double()

In [19]:
a1 = ln(a)

In [12]:
a1.shape

torch.Size([2, 9, 786])

In [6]:
import torch
a = torch.tensor([1,1,0])
b = torch.tensor([0.5,0,1])

In [8]:
torch.max(b-a, torch.tensor ([0.0]))

tensor([0., 0., 1.])

In [5]:
b-a

tensor([-1, -1,  1])

In [9]:
b = torch.full([2, 5], 1.5)

In [10]:
b

tensor([[1.5000, 1.5000, 1.5000, 1.5000, 1.5000],
        [1.5000, 1.5000, 1.5000, 1.5000, 1.5000]])